# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import numpy as np
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kirakira,-10.4544,161.9205,25.00,90,76,1.90,SB,1726414219
1,1,college,64.8569,-147.8028,9.04,86,100,2.06,US,1726414220
2,2,haiku-pauwela,20.9219,-156.3051,23.01,73,75,7.20,US,1726413636
3,3,bredasdorp,-34.5322,20.0403,15.12,74,60,6.81,ZA,1726414222
4,4,betio village,1.3580,172.9211,26.02,74,48,4.34,KI,1726414224


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [23]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    title="Humdity In Each City",
    frame_width = 800,
    frame_height = 600,
    size=np.sqrt(city_data_df['Humidity']) * 1,
    alpha=0.5,
    color="Country"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [27]:
# Narrow down cities that fit criteria and drop any results with null values
city_temp1 = city_data_df.loc[city_data_df['Max Temp']<28,:]
city_temp = city_temp1.loc[city_temp1['Max Temp']>20,:]
city_windy = city_temp.loc[city_temp['Wind Speed']<5.5,:]
city_cloudy = city_windy.loc[city_windy['Cloudiness']==0,:]
city_cloudy

# Drop any rows with null values
city_preferred = city_cloudy.dropna()

# Display sample data
city_preferred

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,bethel,41.3712,-73.4140,22.67,64,0,2.57,US,1726413678
19,19,uturoa,-16.7333,-151.4333,25.13,83,0,3.37,PF,1726414243
22,22,jamestown,42.0970,-79.2353,25.01,57,0,3.09,US,1726413941
86,86,rossosh',51.1209,38.5116,22.14,27,0,4.26,RU,1726414327
107,107,hamilton,39.1834,-84.5333,26.29,53,0,4.12,US,1726413857
128,128,arkhangel'sk,64.5401,40.5433,21.06,52,0,2.00,RU,1726414378
144,144,willard,41.0531,-82.7263,25.80,49,0,3.56,US,1726414397
189,189,choma,-16.8065,26.9531,27.45,17,0,5.22,ZM,1726414450
225,225,sierra vista,31.5545,-110.3037,25.23,51,0,2.57,US,1726414497
241,241,kambove,-10.8764,26.5969,26.11,19,0,1.48,CD,1726414519


### Step 3: Create a new DataFrame called `hotel_df`.

In [28]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_preferred.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,64,
19,uturoa,PF,-16.7333,-151.4333,83,
22,jamestown,US,42.0970,-79.2353,57,
86,rossosh',RU,51.1209,38.5116,27,
107,hamilton,US,39.1834,-84.5333,53,
128,arkhangel'sk,RU,64.5401,40.5433,52,
144,willard,US,41.0531,-82.7263,49,
189,choma,ZM,-16.8065,26.9531,17,
225,sierra vista,US,31.5545,-110.3037,51,
241,kambove,CD,-10.8764,26.5969,19,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apikey":geoapify_key,
    "categories":"available.hotel" 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
rossosh' - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: No hotel found
willard - nearest hotel: No hotel found
choma - nearest hotel: No hotel found
sierra vista - nearest hotel: No hotel found
kambove - nearest hotel: No hotel found
saint-mandrier-sur-mer - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
natchitoches - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
roma - nearest hotel: No hotel found
page - nearest hotel: No hotel found
jinghong - nearest hotel: No hotel found
yarmouth - nearest hotel: No hotel found
tarancon - nearest hotel: No hotel found
chinsali - nearest hotel: No hotel found
staunton - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
15,bethel,US,41.3712,-73.4140,64,No hotel found
19,uturoa,PF,-16.7333,-151.4333,83,No hotel found
22,jamestown,US,42.0970,-79.2353,57,No hotel found
86,rossosh',RU,51.1209,38.5116,27,No hotel found
107,hamilton,US,39.1834,-84.5333,53,No hotel found
128,arkhangel'sk,RU,64.5401,40.5433,52,No hotel found
144,willard,US,41.0531,-82.7263,49,No hotel found
189,choma,ZM,-16.8065,26.9531,17,No hotel found
225,sierra vista,US,31.5545,-110.3037,51,No hotel found
241,kambove,CD,-10.8764,26.5969,19,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    title="Hotel Map",
    frame_width = 800,
    frame_height = 600,
    size=np.sqrt(hotel_df['Humidity']) * 1,
    alpha=0.8,
    hover_cols = ['Hotel Name', 'Country'],
    color="Country"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,_size,Hotel Name)